In [1]:
from datetime import datetime
import sys
import sqlite3
def sieve_of_eratosthenes(start, end):
   
    n = end + 1
    #boolean array
    is_prime = [True] * n
    primes = []

    # Loop  up to the square root of n
    for p in range(2, int(n ** 0.5) + 1):
        # If p is prime, mark all its multiples as not prime
        if is_prime[p]:
            for i in range(p ** 2, n, p):
                 # entery made false if i is Not a prime, else true.
                is_prime[i] = False

    # Append  to the primes list
    for p in range(max(start, 2), n):
        if is_prime[p]:
            primes.append(p)

    return primes


def sieve_of_sundaram(start, end):
    t1 = datetime.utcnow()
    # If Invalid input
    if start < 2:
        start = 2
    if end < start:
        return []

    # Generating  sieve of Sundaram up to n
    n = (end - start) // 2
    sieve = [False] * (n + 1)
    for i in range(1, n+1):
        j = i
        while i+j+2*i*j <= n:
            sieve[i+j+2*i*j] = True
            j += 1

    # Collecting all primes in given range 
    primes = [2*i+1+start for i in range(n+1) if not sieve[i]]
    t2 = datetime.utcnow()
    print((t2 - t1).total_seconds())
    return primes


def sieve_of_atkin(start, end):
    # Initialize the sieve
    sieve = [False] * (end - start + 1)
    # Set up the prime sieve for the sieve of Atkin
    primes = [2, 3, 5]
    sqrt_end = int(end**0.5) + 1
    prime_sieve = [True] * (sqrt_end + 1)
    for i in range(2, sqrt_end):
        if prime_sieve[i]:
            for j in range(i**2, sqrt_end + 1, i):
                prime_sieve[j] = False
            primes.append(i)

    # Apply the sieve of Atkin to find all prime numbers in the range
    for x in range(1, int(sqrt_end)):
        for y in range(1, int(sqrt_end)):
            n = 4*x**2 + y**2
            if n <= end and n >= start and (n % 12 == 1 or n % 12 == 5):
                sieve[n - start] = not sieve[n - start]
            n = 3*x**2 + y**2
            if n <= end and n >= start and n % 12 == 7:
                sieve[n - start] = not sieve[n - start]
            n = 3*x**2 - y**2
            if x > y and n <= end and n >= start and n % 12 == 11:
                sieve[n - start] = not sieve[n - start]

    # Collect all prime numbers in the range
    primes += [i for i in range(start, end+1) if sieve[i - start]]

    # Handle special cases for start=1 and start=2
    if start <= 2:
        primes = [2] + primes
    if start <= 1 and end >= 2:
        primes = [2] + primes[1:]

    return primes
  
conn = sqlite3.connect('primes.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS primes (timestamp TEXT, start INTEGER, end INTEGER, algorithm TEXT, num_primes INTEGER, time_elapsed REAL)')

def get_primes(start, end, algorithm):
    if start < 0 or end < 0 or end < start:
        print('invalid input')
        return
    if algorithm not in ['eratosthenes', 'sundaram', 'atkin']:
        print('invalid input')
        return

    t1 = datetime.utcnow()

    if algorithm == 'eratosthenes':
        primes = sieve_of_eratosthenes(start,end)
    elif algorithm == 'sundaram':
        primes=sieve_of_sundaram(start,end)
    else:
        primes = sieve_of_atkin(start,end)

    t2 = datetime.utcnow()
    time_elapsed = (t2 - t1).total_seconds()

    primes_in_range = [p for p in primes if p >= start and p <= end]
    num_primes = len(primes_in_range)

    # Inserting the Records
    c.execute('INSERT INTO primes VALUES (?, ?, ?, ?, ?, ?)',
              (str(t1), start, end, algorithm, num_primes, time_elapsed))
    conn.commit()

    print(num_primes, primes_in_range)
#Taking User Choice 
print('1.command line')
print('2.program input')
ch=int(input('enter choice '))
if (ch==1):
  start = int(sys.argv[1])
  end = int(sys.argv[2])
  algo = str(sys.argv[3])
elif (ch==2):
  start = int(input('enter start point'))
  end = int(input('enter end point'))
  algo = input('enetr algorithm name ')
else:
  print('invalid input')
#calling function to generate 
get_primes(start, end , algo)
conn.close()

1.command line
2.program input
enter choice 2
enter start point10
enter end point100
enetr algorithm name eratosthenes
21 [11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


This with the Flask Library and Sqlalchemy

In [ ]:
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime


app = Flask('__main__')
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///primes.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class Execution(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    timestamp = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    range_start = db.Column(db.Integer, nullable=False)
    range_end = db.Column(db.Integer, nullable=False)
    algorithm = db.Column(db.String(50), nullable=False)
    num_primes = db.Column(db.Integer, nullable=False)
    time_elapsed = db.Column(db.Float, nullable=False)

    def __repr__(self):
        return f"Execution(timestamp='{self.timestamp}', range_start='{self.range_start}', range_end='{self.range_end}', algorithm='{self.algorithm}', num_primes='{self.num_primes}', time_elapsed='{self.time_elapsed}')"

def sieve_of_eratosthenes(n):
    n = end + 1
    noisprime = [True] * n
    primes = []
    
    # upto square root of n
    for p in range(2, int(n**0.5)+1):
        # If p is prime than  multiples as not prime
        if noisprime[p]:
            for i in range(p**2, n, p):
                is_prime[i] = False

    # Append to the primes list
    for p in range(max(start, 2), n):
        if is_prime[p]:
            primes.append(p)

    return primes

def sieve_of_sundaram(n):
    n = (end - start) // 2
    is_prime = [True] * (n + 1)
    primes = []
    
    # Loop through all possible values of j and k
    for j in range(1, n+1):
        for k in range(j, (n-j)//(2*j)+1):
            index = j + k + 2*j*k
            if index <= n:
                is_prime[index] = False
    
    # Append  to the primes list
    for i in range(len(is_prime)):
        if is_prime[i] and (2*i+1) >= start and (2*i+1) <= end:
            primes.append(2*i+1)

    return primes
def sieve_of_atkin(n):
    is_prime = [False] * (end - start + 1)
    # upto square root of n
    limit = int(end ** 0.5) + 1
    primes = []
    for i in range(2, limit):
        if all(i % j != 0 for j in range(2, int(i ** 0.5) + 1)):
            primes.append(i)
    #  mark all prime numbers between start and end
    for p in primes:
        q = p ** 2
        while q <= end:
            r = q - start % q
            if start % q == 0:
                r = 0
            while r <= end - start:
                is_prime[r] |= True
                r += q
    
    # Append to the primes list
    primes = [i + start for i in range(end - start + 1) if not is_prime[i]]
    return primes

@app.route('/primes/<int:start>/<int:end>/<string:algorithm>')
def get_primes(start, end, algorithm):
    if start < 0 or end < 0 or end < start:
        return jsonify({'error': 'Invalid input'}), 400
    if algorithm not in ['eratosthenes', 'sundaram', 'atkin']:
        return jsonify({'error': 'Invalid algorithm'}), 400
    t1 = datetime.utcnow()
    if algorithm == 'eratosthenes':
        primes = sieve_of_eratosthenes(end)
    elif algorithm == 'sundaram':
        primes = sieve_of_sundaram(end)
    else:
        primes = sieve_of_atkin(end)
    primes_in_range = [p for p in primes if p >= start and p <= end]
    t2 = datetime.utcnow()
    execution_time = (t2 - t1).total_seconds()
    num_primes = len(primes_in_range)
    with app.app_context():
      execution = Execution(range_start=start, range_end=end, algorithm=algorithm, num_primes=num_primes, time_elapsed=execution_time)
      db.session.add(execution)
      db.session.commit()
    return jsonify({'primes': primes_in_range})

if __name__ == '__main__':
  with app.app_context():
    db.create_all()
    app.run(host= '127.0.0.1', port = '5000' ,debug=True)
    executions = Execution.query.all()
    for e in executions:
        print(e)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
